In [1]:
import cv2 as cv
import numpy as np
import os
import sklearn
from sklearn.svm import SVC
from skimage.feature import hog,local_binary_pattern
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV ,StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import pickle

In [2]:
MASK_PATH = './Dataset/mask/'
NO_MASK_PATH = './Dataset/no_mask/'

MASK_PATH_PREPROCESSED = './PreProcessed/mask/'
NO_MASK_PATH_PREPROCESSED = './PreProcessed/no_mask/'



In [3]:
mask_data = []
no_mask_data = []

for i in os.listdir(MASK_PATH_PREPROCESSED):
    path = os.path.join(MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    mask_data.append(img)
   
for i in os.listdir(NO_MASK_PATH_PREPROCESSED):
    path = os.path.join(NO_MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    no_mask_data.append(img)

In [4]:
masked_features = []
no_masked_features = []

numPoints = 24
radius = 8
eps=1e-7

for i in mask_data:
    lbp = local_binary_pattern(i, numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, numPoints + 3),
			range=(0, numPoints + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    masked_features.append(hist)
    
for i in no_mask_data:
    lbp = local_binary_pattern(i, numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, numPoints + 3),
			range=(0, numPoints + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    no_masked_features.append(hist)

In [5]:
df = pd.DataFrame(masked_features)
df['label'] = 1

df1 = pd.DataFrame(no_masked_features)
df1['label'] = 0

merged = pd.concat([df,df1])
merged

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,label
0,0.050720,0.037842,0.014893,0.011963,0.006226,0.005432,0.004517,0.004883,0.005432,0.009155,...,0.008850,0.007690,0.007446,0.008667,0.009644,0.010742,0.021118,0.021240,0.601013,1
1,0.053345,0.037476,0.013611,0.010803,0.010193,0.008362,0.005981,0.007141,0.008545,0.009399,...,0.006165,0.003479,0.006592,0.004150,0.006104,0.010742,0.028320,0.027161,0.614197,1
2,0.052673,0.041992,0.014709,0.013000,0.010559,0.006470,0.006958,0.005981,0.006958,0.009766,...,0.007996,0.005981,0.007812,0.004761,0.006592,0.011047,0.029846,0.029846,0.603088,1
3,0.053162,0.036804,0.013550,0.009766,0.007019,0.007324,0.006165,0.004761,0.005127,0.008057,...,0.008850,0.004272,0.006165,0.005066,0.005371,0.007263,0.028564,0.028259,0.629517,1
4,0.038940,0.030090,0.016296,0.012573,0.008118,0.009033,0.008362,0.008179,0.009216,0.013611,...,0.006470,0.004272,0.004944,0.005188,0.007080,0.010986,0.023254,0.028198,0.594360,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.045959,0.027039,0.020081,0.014282,0.014709,0.016663,0.017090,0.019409,0.018982,0.019104,...,0.013855,0.010132,0.012146,0.006836,0.009399,0.012756,0.016113,0.024597,0.490417,0
638,0.036499,0.027222,0.031433,0.030640,0.023926,0.020203,0.020203,0.017273,0.017761,0.024536,...,0.005615,0.005127,0.008789,0.007812,0.013550,0.023926,0.021729,0.026550,0.411621,0
639,0.044434,0.030701,0.029602,0.021729,0.020203,0.015625,0.015503,0.016479,0.016907,0.015930,...,0.009583,0.007507,0.006958,0.006775,0.008545,0.014771,0.024231,0.025513,0.495972,0
640,0.051575,0.033081,0.016418,0.016785,0.013916,0.014709,0.016479,0.016785,0.017578,0.018372,...,0.012939,0.008606,0.008972,0.006470,0.008423,0.010681,0.016296,0.021851,0.509705,0


In [6]:
y = merged['label']
x = merged.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [7]:
# svm = SVC(kernel='rbf',gamma=0.01, C=0.1)
# svm.fit(X_train,y_train)


# cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['rbf']}  
  
# grid = GridSearchCV(SVC(), param_grid, cv=cross_validation,refit = True, verbose = 3) 
  
# grid.fit(X_train, y_train) 


In [8]:
# print(grid.best_params_)

In [9]:
classifier = SVC(C= 10, gamma= 0.01, kernel= 'rbf')
classifier.fit(X_train, y_train)

SVC(C=10, gamma=0.01)

In [10]:
# best_model = grid.best_estimator_
# y_pred = best_model.predict(X_test)

ytrain_pred = classifier.predict(X_train)
ytest_pred = classifier.predict(X_test)


# print(f"Best parameters found: {grid.best_params_}")
# print(f"Best cross-validation score: {grid.best_score_}")

print(classification_report(y_train, ytrain_pred))
print(classification_report(y_test, ytest_pred))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73       437
           1       0.71      0.78      0.75       427

    accuracy                           0.74       864
   macro avg       0.74      0.74      0.74       864
weighted avg       0.74      0.74      0.74       864

              precision    recall  f1-score   support

           0       0.72      0.69      0.70       205
           1       0.72      0.75      0.74       222

    accuracy                           0.72       427
   macro avg       0.72      0.72      0.72       427
weighted avg       0.72      0.72      0.72       427



In [11]:

matrix_kebingungan = confusion_matrix(ytest_pred, y_test)
print(matrix_kebingungan)

matrix_kebingungan = confusion_matrix(ytrain_pred, y_train)
print(matrix_kebingungan)

[[142  56]
 [ 63 166]]
[[304  94]
 [133 333]]


In [12]:




# img = cv.imread('./b.jpg', cv.IMREAD_GRAYSCALE)
# # img = no_mask_data[-1:]
# # print(img)
# faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
# if not len(faces) < 1:
#     for face_rect in faces:
#         x, y, w, h = face_rect
#         face_image = img[y:y+w, x:x+h]
#         resized = cv.resize(face_image,(128,128))

#         hog_feature = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
#         test_data = pd.DataFrame(hog_feature).T

#         result=best_model.predict(test_data)[0]
#         print(result)
#         if result == 0:
#             result = 'No Mask'
#         else:
#             result = 'Mask'


#         cv.putText(img, result, (100,100), cv.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 1)
#         cv.imshow('',img)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

#         cv.imshow('',resized)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

In [13]:
import pickle
model_filename = './model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_filename}")

NameError: name 'best_model' is not defined